In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

# RQ1

In [186]:
df1 = pd.read_csv('mutation/res/res_misclassification_models_citeseer.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df2 = pd.read_csv('mutation/res/res_misclassification_models_cora.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df3 = pd.read_csv('mutation/res/res_misclassification_models_lastfm.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df4 = pd.read_csv('mutation/res/res_misclassification_models_pubmed.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df = pd.concat([df1, df2, df3, df4], ignore_index=True)


def udf_flag(s):
    check_list = ['clean_mutation', 'clean_deepGini', 'clean_leastConfidence', 'clean_random', 'clean_margin']
    for check in check_list:
        if check in s:
            return 1
    return 0

def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    return s

df['flag'] = df['name'].apply(udf_flag)
df['name'] = df['name'].apply(udf_replace)
df = df[df['flag']==1].reset_index(drop=True)
del df["flag"]
df['Data'] = [i.split('_')[0] for i in df['name']]
df['Model'] = [i.split('_clean_')[0].split('_')[-1] for i in df['name']]
df['Method'] = [i.split('_')[-1] for i in df['name']]
df = df[['Data','Model', 'Method', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70']]

# df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ1.xlsx', index=False)
df.head()

,Data,Model,Method,PFD-10,PFD-20,PFD-30,PFD-40,PFD-50,PFD-60,PFD-70
0,CiteSeer,GCN,AKMGP,0.2981,0.4792,0.6453,0.7057,0.7698,0.8264,0.8717
1,CiteSeer,GCN,Margin,0.2340,0.4038,0.5509,0.6528,0.7283,0.8151,0.8679
2,CiteSeer,GCN,DeepGini,0.2038,0.3472,0.4717,0.5887,0.7057,0.7849,0.8604
3,CiteSeer,GCN,Least Confidence,0.2264,0.3887,0.5396,0.6491,0.7472,0.8151,0.8830
4,CiteSeer,GCN,Random,0.1094,0.2377,0.3132,0.3925,0.4981,0.6151,0.7057


In [187]:
df_tmp = df[(df['Data']=='LastFM')&(df['Model']!='TAGCN')&(df['Model']!='GraphSAGE')]
df_new = df[df['Data']!='LastFM']
df_new = pd.concat([df_new, df_tmp], ignore_index=True)
df_new.head()

,Data,Model,Method,PFD-10,PFD-20,PFD-30,PFD-40,PFD-50,PFD-60,PFD-70
0,CiteSeer,GCN,AKMGP,0.2981,0.4792,0.6453,0.7057,0.7698,0.8264,0.8717
1,CiteSeer,GCN,Margin,0.2340,0.4038,0.5509,0.6528,0.7283,0.8151,0.8679
2,CiteSeer,GCN,DeepGini,0.2038,0.3472,0.4717,0.5887,0.7057,0.7849,0.8604
3,CiteSeer,GCN,Least Confidence,0.2264,0.3887,0.5396,0.6491,0.7472,0.8151,0.8830
4,CiteSeer,GCN,Random,0.1094,0.2377,0.3132,0.3925,0.4981,0.6151,0.7057


In [188]:
df_new = df_new.groupby(['Data', 'Method']).agg('mean').reset_index()

<ipython-input-188-732f46045eba>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_new = df_new.groupby(['Data', 'Method']).agg('mean').reset_index()


In [189]:
df_new

,Data,Method,PFD-10,PFD-20,PFD-30,PFD-40,PFD-50,PFD-60,PFD-70
0,CiteSeer,AKMGP,0.292700,0.509550,0.668625,0.764700,0.829250,0.869475,0.910350
1,CiteSeer,DeepGini,0.191650,0.335900,0.459025,0.584900,0.685175,0.766050,0.847475
2,CiteSeer,Least Confidence,0.214625,0.390725,0.531525,0.642050,0.738725,0.828325,0.896750
3,CiteSeer,Margin,0.218050,0.395050,0.551350,0.667075,0.757200,0.842500,0.897050
4,CiteSeer,Random,0.106125,0.210850,0.305100,0.396275,0.495000,0.589900,0.696875
5,Cora,AKMGP,0.384825,0.703475,0.849575,0.890275,0.920950,0.946750,0.961050
6,Cora,DeepGini,0.253050,0.436200,0.612475,0.732525,0.810475,0.872525,0.928950
7,Cora,Least Confidence,0.264400,0.476225,0.657550,0.773875,0.866350,0.929650,0.958800
8,Cora,Margin,0.245625,0.460125,0.627775,0.759250,0.849375,0.908125,0.953400
9,Cora,Random,0.102050,0.201450,0.293625,0.392750,0.502225,0.594750,0.689350


In [190]:
res = []
M = df_new[['PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70']].to_numpy()

tmp = (M[0]-M[0])/M[0] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[0]-M[1])/M[1] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[0]-M[2])/M[2] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[0]-M[3])/M[3] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[0]-M[4])/M[4] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)
###

tmp = (M[5]-M[5])/M[5] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[5]-M[6])/M[6] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[5]-M[7])/M[7] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[5]-M[8])/M[8] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)


tmp = (M[5]-M[9])/M[9] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

##

tmp = (M[10]-M[10])/M[10] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[10]-M[11])/M[11] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[10]-M[12])/M[12] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[10]-M[13])/M[13] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[10]-M[14])/M[14] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

###

tmp = (M[15]-M[15])/M[15] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[15]-M[16])/M[16] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[15]-M[17])/M[17] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[15]-M[18])/M[18] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

tmp = (M[15]-M[19])/M[19] * 100
tmp = [str(i)[:5] for i in tmp]
res.append(tmp)

df_imp = pd.DataFrame(res, columns=['iPFD-10', 'iPFD-20', 'iPFD-30','iPFD-40','iPFD-50','iPFD-60','iPFD-70'])#创建一个数据表格
df_imp

,iPFD-10,iPFD-20,iPFD-30,iPFD-40,iPFD-50,iPFD-60,iPFD-70
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,52.72,51.69,45.66,30.74,21.02,13.50,7.419
2,36.37,30.41,25.79,19.10,12.25,4.967,1.516
3,34.23,28.98,21.27,14.63,9.515,3.201,1.482
4,175.8,141.6,119.1,92.97,67.52,47.39,30.63
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,52.07,61.27,38.71,21.53,13.63,8.506,3.455
7,45.54,47.71,29.20,15.04,6.302,1.839,0.234
8,56.67,52.88,35.33,17.25,8.426,4.253,0.802
9,277.0,249.2,189.3,126.6,83.37,59.18,39.41


In [191]:
def udf_decimal(s):
    return str(s)[:5]
result = pd.concat([df_new, df_imp], axis=1)
result['PFD-10'] = result['PFD-10'].apply(udf_decimal)
result['PFD-20'] = result['PFD-20'].apply(udf_decimal)
result['PFD-30'] = result['PFD-30'].apply(udf_decimal)
result['PFD-40'] = result['PFD-40'].apply(udf_decimal)
result['PFD-50'] = result['PFD-50'].apply(udf_decimal)
result['PFD-60'] = result['PFD-60'].apply(udf_decimal)
result['PFD-70'] = result['PFD-70'].apply(udf_decimal)

result.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ1_imp.xlsx', index=False)

<ipython-input-191-36824fa1d9cd>:12: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  result.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ1_imp.xlsx', index=False)


# RQ2

In [206]:
df1 = pd.read_csv('mutation/res/res_misclassification_models_citeseer.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df2 = pd.read_csv('mutation/res/res_misclassification_models_cora.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df3 = pd.read_csv('mutation/res/res_misclassification_models_lastfm.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df4 = pd.read_csv('mutation/res/res_misclassification_models_pubmed.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df = pd.concat([df1, df2, df3, df4], ignore_index=True)


def udf_flag(s):
    check_list = ['clean_mutation', 'clean_dnn', 'clean_lgb', 'clean_rf', 'clean_lr']
    for check in check_list:
        if check in s:
            return 1
    return 0

def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'KMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'DNGP')
    s = s.replace('rf', 'RFGP')
    s = s.replace('lr', 'LRGP')
    s = s.replace('lgb', 'LGGP')
    
    return s


df['flag'] = df['name'].apply(udf_flag)
df['name'] = df['name'].apply(udf_replace)
df = df[df['flag']==1].reset_index(drop=True)

del df["flag"]
df['Data'] = [i.split('_')[0] for i in df['name']]
df['Model'] = [i.split('_clean_')[0].split('_')[-1] for i in df['name']]
df['Method'] = [i.split('_')[-1] for i in df['name']]
df = df[['Data','Model', 'Method', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70']]

# df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ2.xlsx', index=False)

df.head(10)

,Data,Model,Method,PFD-10,PFD-20,PFD-30,PFD-40,PFD-50,PFD-60,PFD-70
0,CiteSeer,GCN,DNGP,0.2264,0.4981,0.5925,0.6528,0.6830,0.7321,0.7774
1,CiteSeer,GCN,LGGP,0.2906,0.5057,0.6226,0.6868,0.7283,0.7849,0.8453
2,CiteSeer,GCN,RFGP,0.3132,0.4981,0.6377,0.6981,0.7698,0.8453,0.8906
3,CiteSeer,GCN,LRGP,0.2189,0.5057,0.6340,0.6830,0.7358,0.7774,0.8226
4,CiteSeer,GCN,KMGP,0.2981,0.4792,0.6453,0.7057,0.7698,0.8264,0.8717
5,CiteSeer,GAT,DNGP,0.2263,0.4893,0.6667,0.7217,0.7554,0.7859,0.8440
6,CiteSeer,GAT,LGGP,0.2569,0.4924,0.6850,0.7890,0.8440,0.8624,0.8869
7,CiteSeer,GAT,RFGP,0.2569,0.4893,0.6881,0.7890,0.8257,0.8563,0.8991
8,CiteSeer,GAT,LRGP,0.2355,0.4893,0.6697,0.7554,0.7798,0.8043,0.8440
9,CiteSeer,GAT,KMGP,0.2752,0.4985,0.6667,0.7890,0.8624,0.8930,0.9205


In [207]:
df_new = df.groupby(['Method']).agg('mean').reset_index()
def udf_decimal(s):
    return str(s)[:5]
result = df_new.copy()
result['PFD-10'] = result['PFD-10'].apply(udf_decimal)
result['PFD-20'] = result['PFD-20'].apply(udf_decimal)
result['PFD-30'] = result['PFD-30'].apply(udf_decimal)
result['PFD-40'] = result['PFD-40'].apply(udf_decimal)
result['PFD-50'] = result['PFD-50'].apply(udf_decimal)
result['PFD-60'] = result['PFD-60'].apply(udf_decimal)
result['PFD-70'] = result['PFD-70'].apply(udf_decimal)
result.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ2-2.xlsx', index=False)
result

<ipython-input-207-9f10cba01cd1>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_new = df.groupby(['Method']).agg('mean').reset_index()
<ipython-input-207-9f10cba01cd1>:12: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  result.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ2-2.xlsx', index=False)


,Method,PFD-10,PFD-20,PFD-30,PFD-40,PFD-50,PFD-60,PFD-70
0,DNGP,0.296,0.587,0.721,0.755,0.782,0.813,0.844
1,KMGP,0.344,0.592,0.730,0.799,0.856,0.891,0.924
2,LGGP,0.342,0.635,0.792,0.850,0.881,0.909,0.931
3,LRGP,0.328,0.634,0.774,0.820,0.849,0.879,0.907
4,RFGP,0.352,0.642,0.798,0.852,0.884,0.914,0.940


# RQ3

In [225]:
df1 = pd.read_csv('mutation/res/res_misclassification_models_citeseer.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df2 = pd.read_csv('mutation/res/res_misclassification_models_cora.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df3 = pd.read_csv('mutation/res/res_misclassification_models_lastfm.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df4 = pd.read_csv('mutation/res/res_misclassification_models_pubmed.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df = pd.concat([df1, df2, df3, df4], ignore_index=True)



def udf_flag(s):
    check_list = ['clean']
    for check in check_list:
        if check in s:
            return 1
    return 0

def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'ADNGP')
    s = s.replace('rf', 'ARFGP')
    s = s.replace('lr', 'ALRGP')
    s = s.replace('lgb', 'ALGGP')
    
    s = s.replace('dice', 'DICE')
    s = s.replace('Randomattack_add', 'RAA')
    s = s.replace('Randomattack_flip', 'RAF')
    s = s.replace('Randomattack_remove', 'RAR')
    
    s = s.replace('pgdattack', 'PGD')
    s = s.replace('minmax', 'MMA')
    s = s.replace('nodeembeddinGATtack_remove', 'NEAR')
    s = s.replace('nodeembeddinGATtack_add', 'NEAA')
    
    return s

df['flag'] = df['name'].apply(udf_flag)
df['name'] = df['name'].apply(udf_replace)
df = df[df['flag']==0].reset_index(drop=True)

del df["flag"]
del df["PFD-100"]
del df["PFD-90"]
del df["PFD-80"]
df['Data'] = [i.split('_')[0] for i in df['name']]
df['Model'] = [i.split('_')[1] for i in df['name']]
df['Attack'] = [i.split('_')[2] for i in df['name']]
df['Method'] = [i.split('_')[3] for i in df['name']]

df = df[['Data', 'Model', 'Attack', 'Method', 'PFD-10', 'PFD-20', 'PFD-30', 'PFD-40']]

df.head()

,Data,Model,Attack,Method,PFD-10,PFD-20,PFD-30,PFD-40
0,CiteSeer,GCN,DICE,ADNGP,0.2174,0.4482,0.5719,0.6288
1,CiteSeer,GCN,DICE,ALGGP,0.2308,0.4482,0.6254,0.6856
2,CiteSeer,GCN,DICE,ARFGP,0.2609,0.4515,0.5987,0.6923
3,CiteSeer,GCN,DICE,ALRGP,0.2308,0.4482,0.5819,0.6622
4,CiteSeer,GCN,DICE,AKMGP,0.2508,0.3880,0.5518,0.6722


In [301]:
import pandas as pd

attack_list = sorted(list(set(df['Attack'])))
method_list = sorted(list(set(df['Method'])))

all_res = []

for attack in attack_list:
    tmp_df = df[df['Attack']==attack]
    
    re_dic_10 = dict(zip(method_list, [0]*len(method_list)))
    re_dic_20 = dict(zip(method_list, [0]*len(method_list)))
    re_dic_30 = dict(zip(method_list, [0]*len(method_list)))
    re_dic_40 = dict(zip(method_list, [0]*len(method_list)))
    
    df_tmp = pd.DataFrame(columns=['Attack'])
    
    j = 0
    for _, pdf in tmp_df.groupby(['Data', 'Model']):
        tmp_dic = dict(zip(pdf['Method'], pdf['PFD-10']))
        key = max(tmp_dic, key=tmp_dic.get)
        re_dic_10[key] += 1
        
        tmp_dic = dict(zip(pdf['Method'], pdf['PFD-20']))
        key = max(tmp_dic, key=tmp_dic.get)
        re_dic_20[key] += 1
            
        tmp_dic = dict(zip(pdf['Method'], pdf['PFD-30']))
        key = max(tmp_dic, key=tmp_dic.get)
        re_dic_30[key] += 1
        
        tmp_dic = dict(zip(pdf['Method'], pdf['PFD-40']))
        key = max(tmp_dic, key=tmp_dic.get)
        re_dic_40[key] += 1

    
    df_tmp['Attack'] = [attack]*len(method_list)
    df_tmp['Method'] = method_list
    df_tmp['PFD-10'] = [re_dic_10[i] for i in method_list]
    df_tmp['PFD-20'] = [re_dic_20[i] for i in method_list]
    df_tmp['PFD-30'] = [re_dic_30[i] for i in method_list]
    df_tmp['PFD-40'] = [re_dic_40[i] for i in method_list]
    
    df_tmp['rPFD-10'] = [str(tmp_df[tmp_df['Method']==i]['PFD-10'].mean())[:5] for i in method_list]
    df_tmp['rPFD-20'] = [str(tmp_df[tmp_df['Method']==i]['PFD-20'].mean())[:5] for i in method_list]
    df_tmp['rPFD-30'] = [str(tmp_df[tmp_df['Method']==i]['PFD-30'].mean())[:5] for i in method_list]
    df_tmp['rPFD-40'] = [str(tmp_df[tmp_df['Method']==i]['PFD-40'].mean())[:5] for i in method_list]
    
    all_res.append(df_tmp)

df_rq3 = pd.concat(all_res, ignore_index=True)
df_rq3.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-1.xlsx', index=False)

df_rq3

<ipython-input-301-9bbcbaf3b72a>:52: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df_rq3.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-1.xlsx', index=False)


,Attack,Method,PFD-10,PFD-20,PFD-30,PFD-40,rPFD-10,rPFD-20,rPFD-30,rPFD-40
0,DICE,ADNGP,0,0,1,1,0.277,0.551,0.748,0.808
1,DICE,AKMGP,6,4,1,0,0.300,0.533,0.686,0.777
2,DICE,ALGGP,8,5,6,5,0.302,0.575,0.758,0.838
3,DICE,ALRGP,0,0,3,0,0.284,0.560,0.745,0.817
4,DICE,ARFGP,2,7,5,10,0.304,0.575,0.756,0.846
5,DICE,DeepGini,0,0,0,0,0.212,0.380,0.522,0.632
6,DICE,Least Confidence,0,0,0,0,0.221,0.406,0.560,0.690
7,DICE,Margin,0,0,0,0,0.206,0.388,0.546,0.677
8,DICE,Random,0,0,0,0,0.099,0.197,0.299,0.402
9,MMA,ADNGP,0,1,1,0,0.294,0.570,0.737,0.782


In [303]:
method_list

['ADNGP',
 'AKMGP',
 'ALGGP',
 'ALRGP',
 'ARFGP',
 'DeepGini',
 'Least Confidence',
 'Margin',
 'Random']

In [324]:
df_rq3_2 = pd.DataFrame(columns=['Method'])
pfd_10 = []
pfd_20 = []
pfd_30 = []
pfd_40 = []

rpfd_10 = []
rpfd_20 = []
rpfd_30 = []
rpfd_40 = []
for method in method_list:
    df_rq3_tmp = df_rq3[df_rq3['Method']==method]
    pfd_10.append(df_rq3_tmp['PFD-10'].sum())
    pfd_20.append(df_rq3_tmp['PFD-20'].sum())
    pfd_30.append(df_rq3_tmp['PFD-30'].sum())
    pfd_40.append(df_rq3_tmp['PFD-40'].sum())
    
    rpfd_10.append(df_rq3_tmp['rPFD-10'].astype(float).mean())
    rpfd_20.append(df_rq3_tmp['rPFD-20'].astype(float).mean())
    rpfd_30.append(df_rq3_tmp['rPFD-30'].astype(float).mean())
    rpfd_40.append(df_rq3_tmp['rPFD-40'].astype(float).mean())

df_rq3_2['Method'] = method_list
df_rq3_2['PFD-10'] = pfd_10
df_rq3_2['PFD-20'] = pfd_20
df_rq3_2['PFD-30'] = pfd_30
df_rq3_2['PFD-40'] = pfd_40
df_rq3_2['rPFD-10'] = rpfd_10
df_rq3_2['rPFD-20'] = rpfd_20
df_rq3_2['rPFD-30'] = rpfd_30
df_rq3_2['rPFD-40'] = rpfd_40
df_rq3_2 = df_rq3_2.round({'rPFD-10': 3, 'rPFD-20': 3, 'rPFD-30': 3, 'rPFD-40': 3})
df_rq3_2.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-2.xlsx', index=False)

df_rq3_2

<ipython-input-324-22d7f98d43e5>:33: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df_rq3_2.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-2.xlsx', index=False)


,Method,PFD-10,PFD-20,PFD-30,PFD-40,rPFD-10,rPFD-20,rPFD-30,rPFD-40
0,ADNGP,1,9,11,1,0.294,0.578,0.749,0.800
1,AKMGP,37,22,14,7,0.322,0.569,0.719,0.797
2,ALGGP,37,40,36,33,0.323,0.608,0.776,0.850
3,ALRGP,0,3,7,0,0.304,0.594,0.762,0.828
4,ARFGP,33,34,40,67,0.327,0.610,0.782,0.861
5,DeepGini,0,0,0,0,0.223,0.394,0.536,0.649
6,Least Confidence,0,0,0,0,0.233,0.424,0.586,0.709
7,Margin,0,0,0,0,0.217,0.410,0.575,0.698
8,Random,0,0,0,0,0.098,0.201,0.299,0.399


# RQ4

In [4]:
df1 = pd.read_csv('mutation/res/ratio_res_misclassification_models_citeseer.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df2 = pd.read_csv('mutation/res/ratio_res_misclassification_models_cora.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df3 = pd.read_csv('mutation/res/ratio_res_misclassification_models_lastfm.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df4 = pd.read_csv('mutation/res/ratio_res_misclassification_models_pubmed.csv', names=['name', 'PFD-10', 'PFD-20', 'PFD-30','PFD-40','PFD-50','PFD-60','PFD-70','PFD-80','PFD-90','PFD-100'])
df = pd.concat([df1, df2, df3, df4], ignore_index=True)


def udf_flag(s):
    check_list = ['clean']
    for check in check_list:
        if check in s:
            return 1
    return 0

def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'ADNGP')
    s = s.replace('rf', 'ARFGP')
    s = s.replace('lr', 'ALRGP')
    s = s.replace('lgb', 'ALGGP')
    
    s = s.replace('dice', 'DICE')
    s = s.replace('Randomattack_add', 'RAA')
    s = s.replace('Randomattack_flip', 'RAF')
    s = s.replace('Randomattack_remove', 'RAR')
    
    s = s.replace('pgdattack', 'PGD')
    s = s.replace('minmax', 'MMA')
    s = s.replace('nodeembeddinGATtack_remove', 'NEAR')
    s = s.replace('nodeembeddinGATtack_add', 'NEAA')
    
    return s

df['flag'] = df['name'].apply(udf_flag)
df['name'] = df['name'].apply(udf_replace)
df = df[df['flag']==0].reset_index(drop=True)

del df["flag"]
del df["PFD-100"]
del df["PFD-90"]
del df["PFD-80"]
del df["PFD-70"]
del df["PFD-60"]
del df["PFD-50"]
df['Data'] = [i.split('_')[0] for i in df['name']]
df['Model'] = [i.split('_')[1] for i in df['name']]
df['Attack'] = [i.split('_')[2] for i in df['name']]
df['Method'] = [i.split('_')[-1] for i in df['name']]
df['AttackLevel'] = [i.split('__')[0].split('_')[-1] for i in df['name']]
df = df[(df['AttackLevel']=='0.1')|(df['AttackLevel']=='0.2')|(df['AttackLevel']=='0.3')|(df['AttackLevel']=='0.4')].reset_index(drop=True)
df = df[['Data', 'Model', 'Attack', 'AttackLevel', 'Method', 'PFD-10', 'PFD-20', 'PFD-30', 'PFD-40']]

method_list = sorted(set(df['Method']))
attack_list = sorted(set(df['Attack']))

all_res = []

for attack in attack_list:
    tmp_df_a = df[df['Attack']==attack]
    
    tmp_df_1 = tmp_df_a[tmp_df_a['AttackLevel']=='0.1']
    tmp_df_2 = tmp_df_a[tmp_df_a['AttackLevel']=='0.2']
    tmp_df_3 = tmp_df_a[tmp_df_a['AttackLevel']=='0.3']
    tmp_df_4 = tmp_df_a[tmp_df_a['AttackLevel']=='0.4']
    
    def get_df_tmp(tmp_pdf, name_ratio):
        
        re_dic_10 = dict(zip(method_list, [0] * len(method_list)))
        re_dic_20 = dict(zip(method_list, [0] * len(method_list)))
        re_dic_30 = dict(zip(method_list, [0] * len(method_list)))
        re_dic_40 = dict(zip(method_list, [0] * len(method_list)))

        df_tmp = pd.DataFrame(columns=['Attack'])
        for _, pdf in tmp_pdf.groupby(['Data', 'Model']):
            tmp_dic = dict(zip(pdf['Method'], pdf['PFD-10']))
            key = max(tmp_dic, key=tmp_dic.get)
            re_dic_10[key] += 1

            tmp_dic = dict(zip(pdf['Method'], pdf['PFD-20']))
            key = max(tmp_dic, key=tmp_dic.get)
            re_dic_20[key] += 1

            tmp_dic = dict(zip(pdf['Method'], pdf['PFD-30']))
            key = max(tmp_dic, key=tmp_dic.get)
            re_dic_30[key] += 1

            tmp_dic = dict(zip(pdf['Method'], pdf['PFD-40']))
            key = max(tmp_dic, key=tmp_dic.get)
            re_dic_40[key] += 1


        df_tmp['Attack'] = [attack+'-'+name_ratio] * len(method_list)
        df_tmp['Method'] = method_list
        df_tmp['PFD-10'] = [re_dic_10[i] for i in method_list]
        df_tmp['PFD-20'] = [re_dic_20[i] for i in method_list]
        df_tmp['PFD-30'] = [re_dic_30[i] for i in method_list]
        df_tmp['PFD-40'] = [re_dic_40[i] for i in method_list]

        df_tmp['rPFD-10'] = [str(tmp_pdf[tmp_pdf['Method'] == i]['PFD-10'].mean())[:5] for i in method_list]
        df_tmp['rPFD-20'] = [str(tmp_pdf[tmp_pdf['Method'] == i]['PFD-20'].mean())[:5] for i in method_list]
        df_tmp['rPFD-30'] = [str(tmp_pdf[tmp_pdf['Method'] == i]['PFD-30'].mean())[:5] for i in method_list]
        df_tmp['rPFD-40'] = [str(tmp_pdf[tmp_pdf['Method'] == i]['PFD-40'].mean())[:5] for i in method_list]
        return df_tmp
    
    pdf_1 = get_df_tmp(tmp_df_1, '0.1')
    pdf_2 = get_df_tmp(tmp_df_2, '0.2')
    pdf_3 = get_df_tmp(tmp_df_3, '0.3')
    pdf_4 = get_df_tmp(tmp_df_4, '0.4')
    
    all_res.append(pdf_1)
    all_res.append(pdf_2)
    all_res.append(pdf_3)
    all_res.append(pdf_4)
    
    
    
df_rq4 = pd.concat(all_res, ignore_index=True)
df_rq4.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ4-1.xlsx', index=False)

<ipython-input-4-a0624939cdaa>:132: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df_rq4.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ4-1.xlsx', index=False)


In [18]:
df_rq4['Level'] = [i.split('-')[-1] for i in df_rq4['Attack']]
df_rq4.head()


,Attack,Method,PFD-10,PFD-20,PFD-30,PFD-40,rPFD-10,rPFD-20,rPFD-30,rPFD-40,Level
0,DICE-0.1,ADNGP,0,1,0,0,0.301,0.601,0.773,0.803,0.1
1,DICE-0.1,AKMGP,4,5,2,2,0.327,0.579,0.731,0.802,0.1
2,DICE-0.1,ALGGP,1,1,2,3,0.327,0.617,0.782,0.850,0.1
3,DICE-0.1,ALRGP,0,0,1,0,0.309,0.611,0.772,0.827,0.1
4,DICE-0.1,ARFGP,7,5,7,7,0.331,0.619,0.789,0.860,0.1


In [35]:
method_list = sorted(set(df_rq4['Method']))
df_level_1 = df_rq4[df_rq4['Level']=='0.1']
df_level_2 = df_rq4[df_rq4['Level']=='0.2']
df_level_3 = df_rq4[df_rq4['Level']=='0.3']
df_level_4 = df_rq4[df_rq4['Level']=='0.4']

def get_pdf_level(df_level, level_name):

    df_rq4_2 = pd.DataFrame(columns=['AttackLevel'])

    pfd_10 = []
    pfd_20 = []
    pfd_30 = []
    pfd_40 = []

    rpfd_10 = []
    rpfd_20 = []
    rpfd_30 = []
    rpfd_40 = []
    for method in method_list:
        df_rq4_tmp = df_level[df_level['Method']==method]
        pfd_10.append(df_rq4_tmp['PFD-10'].sum())
        pfd_20.append(df_rq4_tmp['PFD-20'].sum())
        pfd_30.append(df_rq4_tmp['PFD-30'].sum())
        pfd_40.append(df_rq4_tmp['PFD-40'].sum())

        rpfd_10.append(df_rq4_tmp['rPFD-10'].astype(float).mean())
        rpfd_20.append(df_rq4_tmp['rPFD-20'].astype(float).mean())
        rpfd_30.append(df_rq4_tmp['rPFD-30'].astype(float).mean())
        rpfd_40.append(df_rq4_tmp['rPFD-40'].astype(float).mean())

    df_rq4_2['Method'] = method_list
    df_rq4_2['PFD-10'] = pfd_10
    df_rq4_2['PFD-20'] = pfd_20
    df_rq4_2['PFD-30'] = pfd_30
    df_rq4_2['PFD-40'] = pfd_40
    df_rq4_2['rPFD-10'] = rpfd_10
    df_rq4_2['rPFD-20'] = rpfd_20
    df_rq4_2['rPFD-30'] = rpfd_30
    df_rq4_2['rPFD-40'] = rpfd_40
    df_rq4_2 = df_rq4_2.round({'rPFD-10': 3, 'rPFD-20': 3, 'rPFD-30': 3, 'rPFD-40': 3})
    
    df_rq4_2['AttackLevel'] = 'Attack'+'-'+level_name
    
    return df_rq4_2

df_list = [get_pdf_level(df_level_1, '0.1'), get_pdf_level(df_level_2, '0.2'), 
          get_pdf_level(df_level_3, '0.3'), get_pdf_level(df_level_4, '0.4')]

df_rq4_2 = pd.concat(df_list, ignore_index=True)
df_rq4_2.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ4-2.xlsx', index=False)
df_rq4_2

<ipython-input-35-12fea41bb49e>:51: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df_rq4_2.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ4-2.xlsx', index=False)


,AttackLevel,Method,PFD-10,PFD-20,PFD-30,PFD-40,rPFD-10,rPFD-20,rPFD-30,rPFD-40
0,Attack-0.1,ADNGP,0,4,4,0,0.296,0.587,0.743,0.780
1,Attack-0.1,AKMGP,20,18,7,12,0.327,0.583,0.736,0.815
2,Attack-0.1,ALGGP,14,15,21,14,0.325,0.600,0.763,0.833
3,Attack-0.1,ALRGP,2,4,1,0,0.303,0.595,0.753,0.808
4,Attack-0.1,ARFGP,20,15,23,30,0.329,0.602,0.769,0.846
5,Attack-0.1,DeepGini,0,0,0,0,0.217,0.386,0.527,0.640
6,Attack-0.1,Least Confidence,0,0,0,0,0.232,0.420,0.580,0.699
7,Attack-0.1,Margin,0,0,0,0,0.220,0.409,0.566,0.692
8,Attack-0.1,Random,0,0,0,0,0.101,0.199,0.299,0.398
9,Attack-0.2,ADNGP,2,8,6,0,0.287,0.574,0.736,0.781


# RQ5

In [37]:
import pickle

data = pickle.load(open('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/mutation_models/citeseer_gcn/citeseer_gcn_5_53.pkl', 
                        'rb'))



In [38]:
data

{'add_self_loops': True,
 'bias': True,
 'cached': False,
 'improved': True,
 'normalize': False}

In [114]:

citeseer_gat_cover=['citeseer_gat_28_1347A',
                    "citeseer_gat_26_1304A",
                    "citeseer_gat_20_1174A",
                    "citeseer_gat_31_699A",
                    "citeseer_gat_31_1421A",
                    "citeseer_gat_30_1408A",
                    "citeseer_gat_31_1439A",
                    "citeseer_gat_28_645A",
                    "citeseer_gat_25_1289A",
                    "citeseer_gat_24_1262A"]

cora_gcn_cover = ["cora_gcn_28_417A",
                    "cora_gcn_18_263A",
                    "cora_gcn_27_415A",
                    "cora_gcn_8_105A",
                    "cora_gcn_22_322A",
                    "cora_gcn_18_262A",
                    "cora_gcn_26_384A",
                    "cora_gcn_31_468A",
                    "cora_gcn_30_453A",
                    "cora_gcn_30_448A"]

lastfm_graphsage_cover = ["lastfm_graphsage_29_475A",
                        "lastfm_graphsage_26_589A",
                        "lastfm_graphsage_31_419A",
                        "lastfm_graphsage_29_595A",
                        "lastfm_graphsage_28_593A",
                        "lastfm_graphsage_22_581A",
                        "lastfm_graphsage_29_354A",
                        "lastfm_graphsage_30_597A",
                        "lastfm_graphsage_19_515A",
                        "lastfm_graphsage_31_599A"]

pubmed_tagcn_cover = ["pubmed_tagcn_23_582A",
                    "pubmed_tagcn_25_406A",
                    "pubmed_tagcn_25_166A",
                    "pubmed_tagcn_9_554A",
                    "pubmed_tagcn_13_262A",
                    "pubmed_tagcn_31_598A",
                    "pubmed_tagcn_9_314A",
                    "pubmed_tagcn_25_226A",
                    "pubmed_tagcn_29_594A",
                    "pubmed_tagcn_24_464A"]


import os
def get_path(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.pkl'):
                    path_list.append(file_absolute_path)
    return path_list

def get_cover_path(cover_list, path_list):
    res = []
    for i in cover_list:
        for j in path_list:
            if i in j:
                res.append(j)
                break
    return res

path_list = get_path('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/mutation_models')

citeseer_gat_cover_path = get_cover_path(citeseer_gat_cover, path_list)
citeseer_gat_hidden = [int(i.split('/')[-1].split('_')[2]) for i in citeseer_gat_cover_path]
citeseer_gat_epochs = [int(i.split('/')[-1].split('_')[-1].split('.')[0]) for i in citeseer_gat_cover_path]

cora_gcn_cover_path = get_cover_path(cora_gcn_cover, path_list)
cora_gcn_hidden = [int(i.split('/')[-1].split('_')[2]) for i in cora_gcn_cover_path]
cora_gcn_epochs = [int(i.split('/')[-1].split('_')[-1].split('.')[0]) for i in cora_gcn_cover_path]

lastfm_graphsage_cover_path = get_cover_path(lastfm_graphsage_cover, path_list)
lastfm_graphsage_hidden = [int(i.split('/')[-1].split('_')[2]) for i in lastfm_graphsage_cover_path]
lastfm_graphsage_epochs = [int(i.split('/')[-1].split('_')[-1].split('.')[0]) for i in lastfm_graphsage_cover_path]

pubmed_tagcn_cover_path = get_cover_path(pubmed_tagcn_cover, path_list)
pubmed_tagcn_hidden = [int(i.split('/')[-1].split('_')[2]) for i in pubmed_tagcn_cover_path]
pubmed_tagcn_epochs = [int(i.split('/')[-1].split('_')[-1].split('.')[0]) for i in pubmed_tagcn_cover_path]

citeseer_gat_cover_dic_list = [pickle.load(open(i, 'rb')) for i in citeseer_gat_cover_path]
new_citeseer_gat_cover_dic_list = []
for i in range(len(citeseer_gat_cover_dic_list)):
    citeseer_gat_cover_dic_list[i]['hidden_channel'] = citeseer_gat_hidden[i]
    citeseer_gat_cover_dic_list[i]['epochs'] = citeseer_gat_epochs[i]
    new_citeseer_gat_cover_dic_list.append(citeseer_gat_cover_dic_list[i])
    
cora_gcn_cover_dic_list = [pickle.load(open(i, 'rb')) for i in cora_gcn_cover_path]
new_cora_gcn_cover_dic_list = []
for i in range(len(citeseer_gat_cover_dic_list)):
    cora_gcn_cover_dic_list[i]['hidden_channel'] = cora_gcn_hidden[i]
    cora_gcn_cover_dic_list[i]['epochs'] = cora_gcn_epochs[i]
    new_cora_gcn_cover_dic_list.append(cora_gcn_cover_dic_list[i])
    
lastfm_graphsage_cover_dic_list = [pickle.load(open(i, 'rb')) for i in lastfm_graphsage_cover_path]
new_lastfm_graphsage_cover_dic_list = []
for i in range(len(lastfm_graphsage_cover_dic_list)):
    lastfm_graphsage_cover_dic_list[i]['hidden_channel'] = lastfm_graphsage_hidden[i]
    lastfm_graphsage_cover_dic_list[i]['epochs'] = lastfm_graphsage_epochs[i]
    new_lastfm_graphsage_cover_dic_list.append(lastfm_graphsage_cover_dic_list[i])
    
pubmed_tagcn_cover_dic_list = [pickle.load(open(i, 'rb')) for i in pubmed_tagcn_cover_path]
new_pubmed_tagcn_cover_dic_list = []
for i in range(len(citeseer_gat_cover_dic_list)):
    pubmed_tagcn_cover_dic_list[i]['hidden_channel'] = pubmed_tagcn_hidden[i]
    pubmed_tagcn_cover_dic_list[i]['epochs'] = pubmed_tagcn_epochs[i]
    new_pubmed_tagcn_cover_dic_list.append(pubmed_tagcn_cover_dic_list[i])

In [159]:
# new_cora_gcn_cover_dic_list
# new_citeseer_gat_cover_dic_list
# new_lastfm_graphsage_cover_dic_list
# new_pubmed_tagcn_cover_dic_list

orginal_cora_gcn = {'add_self_loops': True, 'bias': True, 'cached': False, 
                    'improved': False, 'normalize': True, 'hidden_channel': 16}

original_citeseer_gat = {'add_self_loops':True, 'bias': True, 'concat': True, 'heads': 4, 
                         'negative_slope': 0.2, 'hidden_channel': 16, 'epochs': 50}

original_lastfm_graphsage = {'bias': True, 'normalize': False, 'hidden_channel':16 , 'epochs': 50}
original_new_pubmed_tagcn = {'normalize': True, 'hidden_channel': 23, 'epochs': 50}


def get_mutation_gcn(dic_mutation, dic):
    res = []
    mutation_list = ['add_self_loops', 'bias', 'cached', 'improved', 'normalize', 'hidden_channel']
    for i in range(len(mutation_list)):
        if i < 5:
            if dic_mutation[mutation_list[i]] != dic[mutation_list[i]]:
                res.append('No')
            else:
                res.append('Yes')
        else:
            value = abs(dic_mutation[mutation_list[i]] - dic[mutation_list[i]])
            res.append(value)       
    return res

def get_mutation_gat(dic_mutation, dic):
    res = []
    mutation_list = ['add_self_loops', 'bias', 'concat', 'heads', 'epochs', 'negative_slope', 'hidden_channel']
    for i in range(len(mutation_list)):
        if i < 6:
            if dic_mutation[mutation_list[i]] != dic[mutation_list[i]]:
                res.append('No')
            else:
                res.append('Yes')
        else:
            value = abs(dic_mutation[mutation_list[i]] - dic[mutation_list[i]])
            res.append(value)       
    return res

def get_mutation_graphsage(dic_mutation, dic):
    res = []
    mutation_list = ['bias', 'normalize', 'hidden_channel', 'epochs']
    for i in range(len(mutation_list)):
        if i < 2:
            if dic_mutation[mutation_list[i]] != dic[mutation_list[i]]:
                res.append('No')
            else:
                res.append('Yes')
        else:
            value = abs(dic_mutation[mutation_list[i]] - dic[mutation_list[i]])
            res.append(value)       
    return res

def get_mutation_tagcn(dic_mutation, dic):
    res = []
    mutation_list = ['normalize', 'hidden_channel', 'epochs']
    for i in range(len(mutation_list)):
        if i < 1:
            if dic_mutation[mutation_list[i]] != dic[mutation_list[i]]:
                res.append('No')
            else:
                res.append('Yes')
        else:
            value = abs(dic_mutation[mutation_list[i]] - dic[mutation_list[i]])
            res.append(value)       
    return res


In [160]:
mutation_list = ['add_self_loops', 'bias', 'cached', 'improved', 'normalize', 'hidden_channel']
res = [get_mutation_gcn(i, orginal_cora_gcn) for i in new_cora_gcn_cover_dic_list]
df_gcn = pd.DataFrame(columns=mutation_list,data=res)
df_gcn

,add_self_loops,bias,cached,improved,normalize,hidden_channel
0,Yes,Yes,No,No,No,12
1,Yes,Yes,Yes,Yes,No,2
2,No,Yes,Yes,Yes,No,11
3,No,Yes,No,No,No,8
4,Yes,Yes,No,Yes,Yes,6
5,Yes,Yes,Yes,Yes,Yes,2
6,Yes,Yes,No,No,Yes,10
7,Yes,Yes,Yes,No,Yes,15
8,Yes,Yes,Yes,No,No,14
9,Yes,Yes,No,No,Yes,14


In [161]:
mutation_list = ['add_self_loops', 'bias', 'concat', 'heads', 'epochs', 'negative_slope', 'hidden_channel']
res = [get_mutation_gat(i, original_citeseer_gat) for i in new_citeseer_gat_cover_dic_list]
df_gat = pd.DataFrame(columns=mutation_list,data=res)
df_gat

,add_self_loops,bias,concat,heads,epochs,negative_slope,hidden_channel
0,Yes,Yes,Yes,Yes,Yes,Yes,12
1,Yes,No,Yes,Yes,Yes,No,10
2,No,No,Yes,No,Yes,No,4
3,Yes,Yes,Yes,Yes,No,Yes,15
4,Yes,Yes,Yes,No,Yes,Yes,15
5,No,Yes,Yes,No,Yes,No,14
6,No,No,Yes,No,Yes,Yes,15
7,No,No,Yes,Yes,No,Yes,12
8,No,Yes,Yes,No,Yes,Yes,9
9,No,Yes,Yes,Yes,Yes,No,8


In [162]:
mutation_list = ['bias', 'normalize', 'hidden_channel', 'epochs']
res = [get_mutation_graphsage(i, original_lastfm_graphsage) for i in new_lastfm_graphsage_cover_dic_list]
df_graphsage = pd.DataFrame(columns=mutation_list,data=res)
df_graphsage

,bias,normalize,hidden_channel,epochs
0,Yes,Yes,13,10
1,Yes,Yes,10,0
2,Yes,Yes,15,15
3,Yes,Yes,13,0
4,Yes,Yes,12,0
5,Yes,Yes,6,0
6,Yes,No,13,20
7,Yes,Yes,14,0
8,Yes,Yes,3,5
9,Yes,Yes,15,0


In [163]:
mutation_list = ['normalize', 'hidden_channel', 'epochs']
res = [get_mutation_tagcn(i, original_new_pubmed_tagcn) for i in new_pubmed_tagcn_cover_dic_list]
df_tagcn = pd.DataFrame(columns=mutation_list,data=res)
df_tagcn

,normalize,hidden_channel,epochs
0,Yes,0,0
1,Yes,2,15
2,Yes,2,35
3,Yes,14,0
4,Yes,10,25
5,Yes,8,0
6,Yes,14,20
7,Yes,2,30
8,Yes,6,0
9,Yes,1,10


# APFD

## RQ1

In [15]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

def get_clean(s):
    if 'clean' in s:
        return True
    else:
        return False
    
def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'ADNGP')
    s = s.replace('rf', 'ARFGP')
    s = s.replace('lr', 'ALRGP')
    s = s.replace('lgb', 'ALGGP')
    
    s = s.replace('dice', 'DICE')
    s = s.replace('Randomattack_add', 'RAA')
    s = s.replace('Randomattack_flip', 'RAF')
    s = s.replace('Randomattack_remove', 'RAR')
    
    s = s.replace('pgdattack', 'PGD')
    s = s.replace('minmax', 'MMA')
    s = s.replace('nodeembeddinGATtack_remove', 'NEAR')
    s = s.replace('nodeembeddinGATtack_add', 'NEAA')
    
    return s

def get_value(s):
    return str(s)[:5]

def get_data_name(s):
    return s.split('_')[0]

def get_model_name(s):
    return s.split('_')[1]

def get_value(s):
    return str(s)[:5]

df = pd.read_csv('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/apfd.csv', 
                 names=['name', 'DGGP', 'KMGP', 'LGGP', 'LRGP', 
                        'RFGP', 'DeepGini', 'LeastConfidence', 'Margin', 'Random'])

df['flag'] = df['name'].apply(get_clean)
df['name'] = df['name'].apply(udf_replace)
df['Data'] = df['name'].apply(get_data_name)
df['Method'] = df['name'].apply(get_model_name)

cols = ['DGGP', 'KMGP', 'LGGP', 'LRGP', 'RFGP', 'DeepGini', 'LeastConfidence', 'Margin', 'Random']
for i in cols:
    df[i] = df[i].apply(get_value)

df = df[df['flag']==True]
del df['flag']
del df['Random']

df = df[['Data', 'Method', 'KMGP', 'DeepGini', 'LeastConfidence', 'Margin']]
df = df.sort_values(by=['Data', 'Method'])

df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ1-3.xlsx', index=False)

df

<ipython-input-15-48555dcb3e80>:77: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ1-3.xlsx', index=False)


,Data,Method,KMGP,DeepGini,LeastConfidence,Margin
7,CiteSeer,GAT,0.741,0.638,0.675,0.681
0,CiteSeer,GCN,0.706,0.643,0.672,0.672
21,CiteSeer,GraphSAGE,0.748,0.611,0.665,0.682
14,CiteSeer,TAGCN,0.733,0.627,0.672,0.680
37,Cora,GAT,0.826,0.720,0.752,0.743
28,Cora,GCN,0.812,0.726,0.744,0.728
55,Cora,GraphSAGE,0.803,0.710,0.735,0.718
46,Cora,TAGCN,0.819,0.693,0.741,0.734
72,LastFM,GAT,0.808,0.585,0.638,0.653
64,LastFM,GCN,0.771,0.691,0.737,0.750


## RQ2

In [26]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

def get_clean(s):
    if 'clean' in s:
        return True
    else:
        return False
    
def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'ADNGP')
    s = s.replace('rf', 'ARFGP')
    s = s.replace('lr', 'ALRGP')
    s = s.replace('lgb', 'ALGGP')
    
    s = s.replace('dice', 'DICE')
    s = s.replace('Randomattack_add', 'RAA')
    s = s.replace('Randomattack_flip', 'RAF')
    s = s.replace('Randomattack_remove', 'RAR')
    
    s = s.replace('pgdattack', 'PGD')
    s = s.replace('minmax', 'MMA')
    s = s.replace('nodeembeddinGATtack_remove', 'NEAR')
    s = s.replace('nodeembeddinGATtack_add', 'NEAA')
    
    return s

def get_value(s):
    return str(s)[:5]

def get_data_name(s):
    return s.split('_')[0]

def get_model_name(s):
    return s.split('_')[1]

def get_value(s):
    return str(s)[:5]

df = pd.read_csv('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/apfd.csv', 
                 names=['name', 'DNGP', 'KMGP', 'LGGP', 'LRGP', 
                        'RFGP', 'DeepGini', 'LeastConfidence', 'Margin', 'Random'])

df['flag'] = df['name'].apply(get_clean)
df['name'] = df['name'].apply(udf_replace)
df['Data'] = df['name'].apply(get_data_name)
df['Method'] = df['name'].apply(get_model_name)

cols = ['DNGP', 'KMGP', 'LGGP', 'LRGP', 'RFGP', 'DeepGini', 'LeastConfidence', 'Margin', 'Random']
for i in cols:
    df[i] = df[i].apply(get_value)

df = df[df['flag']==True]
del df['flag']
del df['Random']

df = df[['Data', 'Method', 'DNGP', 'LGGP', 'RFGP', 'LRGP', 'KMGP']]
df = df.sort_values(by=['Data', 'Method'])

df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ2-3.xlsx', index=False)

df

<ipython-input-26-208b59ae1398>:77: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ2-3.xlsx', index=False)


,Data,Method,DNGP,LGGP,RFGP,LRGP,KMGP
7,CiteSeer,GAT,0.682,0.722,0.724,0.690,0.741
0,CiteSeer,GCN,0.642,0.685,0.710,0.672,0.706
21,CiteSeer,GraphSAGE,0.672,0.718,0.733,0.693,0.748
14,CiteSeer,TAGCN,0.683,0.714,0.723,0.703,0.733
37,Cora,GAT,0.785,0.820,0.827,0.784,0.826
28,Cora,GCN,0.768,0.808,0.803,0.778,0.812
55,Cora,GraphSAGE,0.769,0.765,0.811,0.793,0.803
46,Cora,TAGCN,0.760,0.812,0.799,0.770,0.819
72,LastFM,GAT,0.812,0.819,0.827,0.805,0.808
64,LastFM,GCN,0.825,0.820,0.827,0.810,0.771


## RQ3 

In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

def get_clean(s):
    if 'clean' in s:
        return True
    else:
        return False
    
def udf_replace(s):
    s = s.replace('citeseer', 'CiteSeer')
    s = s.replace('cora', 'Cora')
    s = s.replace('pubmed', 'PubMed')
    s = s.replace('lastfm', 'LastFM')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('gcn', 'GCN')
    s = s.replace('gat', 'GAT')
    s = s.replace('graphsage', 'GraphSAGE')
    s = s.replace('tagcn', 'TAGCN')
    s = s.replace('mutation', 'AKMGP')
    s = s.replace('deepGini', 'DeepGini')
    s = s.replace('leastConfidence', 'Least Confidence')
    s = s.replace('random', 'Random')
    s = s.replace('margin', 'Margin')
    
    s = s.replace('dnn', 'ADNGP')
    s = s.replace('rf', 'ARFGP')
    s = s.replace('lr', 'ALRGP')
    s = s.replace('lgb', 'ALGGP')
    
    s = s.replace('dice', 'DICE')
    s = s.replace('Randomattack_add', 'RAA')
    s = s.replace('Randomattack_flip', 'RAF')
    s = s.replace('Randomattack_remove', 'RAR')
    
    s = s.replace('pgdattack', 'PGD')
    s = s.replace('minmax', 'MMA')
    s = s.replace('nodeembeddinGATtack_remove', 'NEAR')
    s = s.replace('nodeembeddinGATtack_add', 'NEAA')
    
    return s

def get_value(s):
    return str(s)[:5]

def get_data_name(s):
    return s.split('_')[0]

def get_model_name(s):
    return s.split('_')[1]

def get_value(s):
    return str(s)[:5]

def get_attack(s):
    return s.split('_')[2]

df = pd.read_csv('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/apfd.csv', 
                 names=['name', 'DNGP', 'KMGP', 'LGGP', 'LRGP', 
                        'RFGP', 'DeepGini', 'Least Confidence', 'Margin', 'Random'])

df['flag'] = df['name'].apply(get_clean)
df['name'] = df['name'].apply(udf_replace)
df['Data'] = df['name'].apply(get_data_name)
df['Method'] = df['name'].apply(get_model_name)
df['Attack'] = df['name'].apply(get_attack)

# cols = ['DNGP', 'KMGP', 'LGGP', 'LRGP', 'RFGP', 'DeepGini', 'Least Confidence', 'Margin', 'Random']
# for i in cols:
#     df[i] = df[i].apply(get_value)

del df['Random']
df = df[df['flag']==False]

df = df[['Attack','DNGP', 'KMGP', 'LGGP', 'LRGP', 'RFGP', 'DeepGini', 'Least Confidence', 'Margin']]
df



df = df.groupby('Attack').agg('mean').reset_index()

cols = ['DNGP', 'KMGP', 'LGGP', 'LRGP', 'RFGP', 'DeepGini', 'Least Confidence', 'Margin']
for i in cols:
    df[i] = df[i].apply(get_value)

df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-3.xlsx', index=False)

df

<ipython-input-49-9fa30e84d13b>:88: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  df.to_excel('/Users/yinghua.li/Documents/Pycharm/GNNEST/mutation/res/RQ3-3.xlsx', index=False)


,Attack,DNGP,KMGP,LGGP,LRGP,RFGP,DeepGini,Least Confidence,Margin
0,DICE,0.738,0.742,0.768,0.749,0.769,0.652,0.682,0.672
1,MMA,0.728,0.770,0.757,0.733,0.774,0.669,0.702,0.698
2,NEAA,0.760,0.757,0.796,0.780,0.795,0.664,0.697,0.682
3,NEAR,0.710,0.777,0.800,0.785,0.802,0.662,0.698,0.688
4,PGD,0.748,0.775,0.777,0.749,0.791,0.668,0.704,0.704
5,RAA,0.739,0.749,0.770,0.751,0.771,0.653,0.682,0.669
6,RAF,0.741,0.751,0.771,0.754,0.772,0.648,0.679,0.667
7,RAR,0.738,0.763,0.779,0.757,0.780,0.657,0.691,0.685


In [34]:
len(df)

96